Erst mal die Daten in eine richtige Datenbank importieren!

In [11]:
import psycopg2
import numpy as np
import pandas as pd
import io
import sqlite3
from sqlalchemy import create_engine, Column, Integer, ARRAY, MetaData, Table, Text
from sqlalchemy.dialects.postgresql import ARRAY as PG_ARRAY
from psycopg2.extensions import register_adapter, AsIs

# Make array from the binary data in the database
def _convert_array(text):
	out = io.BytesIO(text)
	out.seek(0)
	return np.load(out, allow_pickle=True)

# Make sqlite3 binary from the given array (probably for saving the array in the database)
def _adapt_array(arr):
	out = io.BytesIO()
	np.save(out, arr)
	out.seek(0)
	return sqlite3.Binary(out.read())

sqlite3.register_adapter(np.ndarray, _adapt_array)
sqlite3.register_converter("ARRAY", _convert_array)


# Custom adapter function for postgre
def adapt_numpy_ndarray(numpy_array):
    return AsIs(list(numpy_array))

# Register the postgre-adapter
register_adapter(np.ndarray, adapt_numpy_ndarray)


# Database connection parameters
dbname = 'bathunting'
user = 'python'
password = 'python_password'
host = 'localhost'
port = '5432' 
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

metadata = MetaData()
my_table = Table('batcall', metadata,
                 Column('call', Integer, primary_key=True),
                 Column('target', Integer),
                 Column('id', Integer),
                 Column('bat', Text),
                 Column('my_array', PG_ARRAY(Integer)))

metadata.create_all(engine)


# Connect to the database
pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

# Create a cursor object
pg_cur = pg_conn.cursor()

# Execute a query
#pg_cur.execute('CREATE TABLE batcall(call serial primary key, target int NOT NULL,id int, bat text NOT NULL, arr int[] NOT NULL )')

lite_conn = sqlite3.connect('batcallsv14.db', detect_types=True)
lite_cursor = lite_conn.cursor()
query = f"SELECT call, target, id, bat, arr FROM batcalls"
df = pd.read_sql_query(query, lite_conn)
lite_conn.close()


df['arr'] = df['arr'].apply(lambda x: x.tolist())
df.to_sql('batcall', engine, index=False, if_exists='replace')


# Close the cursor and connection
pg_cur.close()
pg_conn.close()

In [13]:
import numpy as np
import pandas as pd
import io
import sqlite3
lite_conn = sqlite3.connect('batcallsv14.db', detect_types=True)
lite_cursor = lite_conn.cursor()
query = f"SELECT count(*) FROM batcalls"
df = pd.read_sql_query(query, lite_conn)
lite_conn.close()
print(df)

   count(*)
0     51719
